In [1]:
# use yahoo finance as data source
import yfinance as yf

# Yili Group from Shanghai Stock Exchange (ticker: 600887), 2 months period, and close price
# start date inclusive; end date exclusive
data = yf.Ticker('600887.SS').history(start = '2021-08-01', end = '2021-10-01')[['Close']]

In [2]:
# method 1: pandas.Dataframe.ewm
# 26-period exponentially weighted mean  
EMA_26 = data.Close.ewm(span=26, adjust=False, min_periods=26).mean()

# 12-period exponentially weighted mean  
EMA_12 = data.Close.ewm(span=12, adjust=False, min_periods=12).mean()

# macd equals 26-period EMA plus 12-period EMA
macd = EMA_12 - EMA_26

# the signal line is 9 periods exponentially moving average of macd
macd_s = macd.ewm(span=9, adjust=False, min_periods=9).mean()

# the histogram is macd minus the signal line
macd_h = macd - macd_s

# integrate the data together
data['EMA_26'] = data.index.map(EMA_26)
data['EMA_12'] = data.index.map(EMA_12)
data['macd'] = data.index.map(macd)
data['macds'] = data.index.map(macd_s)
data['macdh'] = data.index.map(macd_h)

In [3]:
# method 2: pandas_ta
import pandas_ta as ta
data.ta.macd(close='Close', fast = 12, slow=26, signal = 9, append=True)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
Date,,,
2021-08-02,NaN,NaN,NaN
2021-08-03,NaN,NaN,NaN
2021-08-04,NaN,NaN,NaN
2021-08-05,NaN,NaN,NaN
2021-08-06,NaN,NaN,NaN
2021-08-09,NaN,NaN,NaN
2021-08-10,NaN,NaN,NaN
2021-08-11,NaN,NaN,NaN
2021-08-12,NaN,NaN,NaN


In [4]:
# method 3: stockstats
from stockstats import StockDataFrame as Sdf
import pandas as pd
sdata = yf.Ticker('600887.SS').history(start = '2021-08-01', end = '2021-10-01')[['Close']]
sdata = Sdf.retype(sdata)

In [5]:
# integrate all three methods
data2 = pd.concat([data, sdata[['macd','macds','macdh']]],join='inner', axis=1) 

# export
data2.to_excel('data2.xlsx')